In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

In [160]:
gun_violence_db = pd.read_csv('/home/onyxia/gun_violence/rendu_final/data/gun_violence_db.csv')
gun_violence_db.sample(5)

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident
149753,149753,670790,2016-10-05,District of Columbia,Washington,"8th Street and D Street, NW",0,0,http://www.gunviolencearchive.org/incident/670790,https://twitter.com/DCPoliceDept/status/783843...,False,1.0,{0: 'Unknown'},{0: 'Unknown'},"['Non-Shooting Incident', 'Car-jacking']",38.8948,NaN,-77.0230,1.0,NaN,NaN,"{1: 'Adult 18+', 2: 'Adult 18+'}","{1: 'Male', 2: 'Male'}",NaN,NaN,"{0: 'Unharmed', 1: 'Unharmed', 2: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect', 2: 'Subjec...",NaN,NaN,NaN,0,0,0,0,0
128521,128521,569732,2016-05-30,Georgia,Rome,Broad Street and Third Avenue,0,0,http://www.gunviolencearchive.org/incident/569732,http://www.northwestgeorgianews.com/rome/news/...,False,14.0,{0: 'Unknown'},{0: '9mm'},"['Non-Shooting Incident', 'Gun(s) stolen from ...",34.2537,NaN,-85.1729,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,52.0,0,0,0,0,0
156296,156296,699420,2016-11-13,Indiana,Indianapolis,5150 West 38th Street,0,1,http://www.gunviolencearchive.org/incident/699420,http://wishtv.com/2016/11/13/one-injured-in-ni...,False,7.0,{0: 'Unknown'},{0: 'Unknown'},"['Shot - Wounded/Injured', 'Bar/club incident ...",39.8240,Limelight Nightclub,-86.2477,1.0,Man shot at 2:0am in bar;,NaN,"{0: 'Adult 18+', 1: 'Adult 18+'}",{0: 'Male'},NaN,NaN,"{0: 'Injured', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",NaN,94.0,33.0,0,0,0,0,0
77039,77039,365102,2015-06-27,New York,Plattekill,400 block of Crescent Avenue,1,0,http://www.gunviolencearchive.org/incident/365102,http://www.midhudsonnews.com/News/2015/Decembe...,False,19.0,{0: 'Unknown'},{0: 'Shotgun'},"['Shot - Dead (murder, accidental, suicide)', ...",41.7021,NaN,-74.0380,1.0,"perp stabbed 3 people, shot by vics fam member...",{0: '25'},{0: 'Adult 18+'},{0: 'Male'},0::Rodshan Pittman,{0: 'Friends'},{0: 'Killed'},{0: 'Subject-Suspect'},['http://www.dailyfreeman.com/general-news/201...,103.0,39.0,1,1,0,0,1
127010,127010,563099,2016-05-21,Illinois,Chicago,900 block of North Homan Avenue,0,1,http://www.gunviolencearchive.org/incident/563099,https://www.dnainfo.com/chicago/20160521/humbo...,False,7.0,NaN,NaN,NaN,41.8974,NaN,-87.7118,NaN,Humboldt Park - man shot while driving. lower ...,{0: '33'},{0: 'Adult 18+'},{0: 'Male'},NaN,NaN,{0: 'Injured'},{0: 'Victim'},NaN,10.0,5.0,0,0,0,0,0


In [ ]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [ ]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']

In [163]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [ ]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214 

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

Merge des deux bases

In [ ]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

In [ ]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

In [ ]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

In [ ]:
pd.set_option('display.max_columns', None)
merge_geo.sample()

# Stats descriptives sur les indicateurs socio-économiques des comtés
## Revenu médian

In [ ]:
print("I. Statistiques descriptives sur les indicateurs socio-économiques des comtés \n")
print("Le revenu médian :")

In [ ]:
total_db_geo.plot(aspect = 1, figsize = (12,5), column = "med_h_income_year_2013", legend = True, cmap = "plasma", linewidth = 0.1)
plt.title("Revenu médian annuel par foyer en 2013")
plt.show()

Ces représentations sont difficilement lisibles car il y a beaucoup de comtés. Une solution est de classer les comtés et de les colorier en fonction de leur classement, en définissant par exemple 4 quantiles. 

In [ ]:
print("Ces représentations sont difficilement lisibles car il y a beaucoup de comtés. \nUne solution est de classer les comtés et de les colorier en fonction de leur classement, en définissant par exemple 4 quantiles. ")

In [ ]:
total_db_geo.plot(aspect = 1, figsize = (15,15), column = "med_h_income_year_2013", legend = True, 
scheme = "Quantiles", k = 4, linewidth = 0.1, legend_kwds={'loc': 'lower left'}) #scheme permet de découper en quantiles
plt.title("Revenu médian annuel par foyer en 2013")
plt.show()

Cette représentation a l'avantage de permettre de distinguer rapidement des groupes de comtés. Cependant, il faut garder en tête que la dernière catégorie recouvre beaucoup de valeurs différentes (du simple au double).

On voit sur cette carte que les comtés du Sud-Ouest sont ceux avec le revenu médian le plus faible, alors que les plus rivhes se situent surtout au Nord.

In [ ]:
print("Cette représentation a l'avantage de permettre de distinguer rapidement des groupes de comtés. \nCependant, il faut garder en tête que la dernière catégorie recouvre beaucoup de valeurs différentes (du simple au double).")
print("On voit sur cette carte que les comtés du Sud-Ouest sont ceux avec le revenu médian le plus faible, alors que les plus riches se situent surtout au Nord.")

In [ ]:
sns.set(style="ticks")
x = total_db_geo["med_h_income_year_2013"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize = (10, 5), gridspec_kw={"height_ratios": (.15, .85)}) #on créé deux graphiques

sns.boxplot(x=x, ax=ax_box) #on génère les graphiques à leur place
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist) #on supprime les côtés
sns.despine(ax=ax_box, left=True)

plt.suptitle("Distribution du revenu annuel médian par foyer en 2013")
ax_hist.set_xlabel('Revenu annuel médian par foyer en 2013')

In [ ]:
print("La distribution du revenu annuel médian par foyer en 2013 semble suivre une loi log-normale, car elle ressemble à une normale avec une plus grande queue à droite. \nLe box-plot semble confirmer cela : les 'valeurs aberrantes' ne sont présentes qu'à droite de la distribution.\n")


La distribution du revenu annuel médian par foyer en 2013 semble suivre une loi log-normale, car elle ressemble à une normale avec une plus grande queue à droite. Le box-plot semble confirmer cela : les "valeurs aberrantes" ne sont présentes qu'à droite de la distribution.

## Proportion de détenteurs d'un bachelor

In [ ]:
print("Proportion de détenteurs d'un bachelor : ")

In [ ]:
total_db_geo.plot(aspect = 1, figsize = (15,15), column = "bachelors_deg_year_2013", legend = True, scheme = "QUANTILES", 
k=4, linewidth = 0.1, legend_kwds={'loc': 'lower left'})
plt.title("Proportion de détenteurs d'un bachelor en 2013")
plt.show()

A nouveau, le Nord-Est est privilégié, avec la plus grosse part de détententeurs de bachelors. Le Sud-Est est encore une fois défavorisé, avec preque tout le premier quantile situé à cet endroit.

In [ ]:
print("A nouveau, le Nord-Est est privilégié, avec la plus grosse part de détententeurs de bachelors. \nLe Sud-Est est encore une fois défavorisé, avec la quasi-totalité du premier quantile située à cet endroit.")

In [ ]:
sns.set(style="ticks")
x = total_db_geo["bachelors_deg_year_2013"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize = (10, 5), gridspec_kw={"height_ratios": (.15, .85)})

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution de la proportion de diplômés de bachelor en 2013")
ax_hist.set_xlabel('Proportion de diplômés de bachelor en 2013')

On obtient encore une fois une courbe normale biaisé vers la droite de la distribution. Les valeurs aberrantes sont toutes situées de ce côté de la distribution. On voit bien sur le boxplot les inégalités très importantes qui peut exister entre les comtés. Cela est assez logique : le comté étant un découpage relativement fin du territoire américain, il n'est pas étonnant de grosses variations à cette échelle.

## Bénéficiaires du programme SNAP

In [ ]:
print("On obtient encore une fois une courbe normale biaisée vers la droite de la distribution. Les valeurs aberrantes sont toutes situées de ce côté de la distribution. \nOn voit bien sur le boxplot les inégalités très importantes qui peuvent exister entre les comtés. Cela est assez logique : le comté étant un découpage relativement fin du territoire américain, il n'est pas étonnant que de grosses variations existent à cette échelle.\n")
print("Bénéficiaires du programme SNAP :")

In [ ]:
total_db_geo["snap_share_year_2013"] = total_db_geo["snap_beneficiaries_year_2013"] / total_db_geo["resident_pop_year_2013"] / 1000
total_db_geo.plot(aspect = 1, figsize = (15,15), column = "snap_share_year_2013", legend = True, scheme = "QUANTILES", 
k=4, linewidth = 0.1, legend_kwds={'loc': 'lower left'})
plt.title("Proportion de bénéficiaires du programme SNAP en 2013 en %")
plt.show()

On retrouve la distinction Nord-Est privilégié / Sud-Est défavorisé ici encore : les taux de bénéficaires les plus bas sont situés au Nord-Est et les plus importants dnas le Sud, et en particulier le Sud-Est. Le Nord et le centre du pays font partie des deux premiers quartiles, alors que le Nord-Ouest fait également partie des territoires avec la plus grande part de bénéficiaires.

In [ ]:
print("On retrouve la distinction Nord-Est privilégié / Sud-Est défavorisé ici encore : les taux de bénéficaires les plus bas sont situés au Nord-Est et les plus importants dnas le Sud, et en particulier le Sud-Est. \nLe Nord et le centre du pays font partie des deux premiers quartiles, alors que le Nord-Ouest fait également partie des territoires avec la plus grande part de bénéficiaires.")

In [ ]:

sns.set(style="ticks")
x = total_db_geo["snap_share_year_2013"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution de la proportion de bénéficiaire du SNAP en 2013 en %")
ax_hist.set_xlabel('Proportion de bénéficiaire du SNAP en 2013 en %')

On retrouve globalement la même forme de distribution, mais un peu plus étalée que les deux vues précédemment. Encore une fois, la courbe en cloche est biaisée vers la droite, où se situent toutes les valeurs aberrantes. 

## Taux de chômage

In [ ]:
print("On retrouve globalement la même forme de distribution, mais un peu plus étalée que les deux vues précédemment. Encore une fois, la courbe en cloche est biaisée vers la droite, où se situent toutes les valeurs aberrantes. \n")
print("Taux de chômage :")

In [ ]:
total_db_geo.plot(aspect = 1, figsize = (15,15), column = "unemp_rate_year_2013", legend = True, scheme = "QUANTILES", k=4, 
linewidth = 0.1, legend_kwds={'loc': 'lower left'})
plt.title("Taux de chômage en 2013")
plt.show()

La répartition géographique du taux de chômage semble totalement différente des autres : toute la côte Ouest est dans le dernier quartile, et l'Est aussi, alors que c'est le centre qui a le taux de chômage le plus bas. Certaines zones connues pour leur désindustrialisation sont dans le dernier quartile, comme la région de Détroit par exemple.

In [ ]:
print("La répartition géographique du taux de chômage semble totalement différente des autres : toute la côte Ouest ainsi que l'Est sont dans le dernier quartile, alors que c'est le centre qui a le taux de chômage le plus faible. \nCertaines zones connues pour leur désindustrialisation sont dans le dernier quartile, comme la région de Détroit par exemple.")

In [ ]:
sns.set(style="ticks")
x = total_db_geo["unemp_rate_year_2013"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution du taux de chômage en 2013")
ax_hist.set_xlabel('Taux de chômage en 2013')

La distribution a la forme que nous avons vu déjà plusieurs fois. On remarque que les valuers aberrantes sont de plus en plus espacées au fur et à mesure qu'elles augmentent.

## Vote pour les démocrates à l'élection présidentielle de 2016

In [ ]:
print("La forme de la distribution est de nouveau similaires à celles vues précédemments. \nOn remarque que les valuers aberrantes sont de plus en plus espacées au fur et à mesure qu'elles augmentent.\n")
print("Vote pour les démocrates à l'élection présidentielle de 2016")

In [ ]:
total_db_geo.plot(aspect = 1, figsize = (12,6), column = "per_dem", legend = True, linewidth = 0.1, cmap = "RdBu")
plt.title("Part des votes pour les démocrates en 2016")
plt.show()

Cette représentation a le défaut de laisser penser que les républicains ont largement gagné le vote de 2016, puisque la carte est en grande majorité rouge. Pour remédier à cela, il faut faire une représentation qui pondère le vote par la population résidente de 2016.

In [ ]:
print("Cette représentation a le défaut de laisser penser que les républicains ont largement gagné le vote de 2016, puisque la carte est en grande majorité rouge. \nPour remédier à cela, il faut faire une représentation qui pondère le vote par la population résidente de 2016.")

In [ ]:
total_db_geo = total_db_geo.to_crs(4329)
total_db_geo['centroid'] = total_db_geo['geometry'].centroid

# Create a GeoDataFrame with centroids
gdf_centroids = gpd.GeoDataFrame(total_db_geo[['resident_pop_year_2016', "per_dem", "centroid"]], geometry=total_db_geo['centroid'])

# Plot
# Plot polygons
fig, ax = plt.subplots(figsize=(12, 6))
total_db_geo.plot(ax=ax, color='lightgray', alpha=0.7, edgecolor='white', linewidth=0.1)

# Plot centroids as dots with color and size based on variables
sc = ax.scatter(
    gdf_centroids['centroid'].x,
    gdf_centroids['centroid'].y,
    c=gdf_centroids['per_dem'],
    cmap='RdBu',
    s=gdf_centroids['resident_pop_year_2016']*0.5,
    alpha=0.7,
    marker='o',
    edgecolors='black',  # Add black edges for better visibility
    linewidths=1,        # Adjust linewidth of the edges
)

# Set plot title and labels
plt.title('Représentation du votes démocrates en 2016, pondéré par la population de 2016')

# Add colorbar
cbar = plt.colorbar(sc, label='Part du vote démocrate')

# Show the plot
plt.show()

Même si la carte n'est pas forcément évidente à lire à cause des chevauchements des cercles (dont la taille dépend de la population résidente), on voit que le bleu et le rouge (et donc les électeurs démocrates et républicains) sont relativement égaux sur le territoire, ce qui est plus fidèle au résultat de l'élection. La Californie et le Nord-Est (qui sont souvent favorisés dans les statistiques précédentes) sont plutôt démocrates, alors que le Sud-Est, plus défavorisé, penche plutôt en faveur des républicains

In [ ]:
print("Même si la carte n'est pas forcément évidente à lire à cause des chevauchements des cercles (dont la taille dépend de la population résidente), on voit que le bleu et le rouge (et donc les électeurs démocrates et républicains) sont relativement égaux sur le territoire, ce qui est plus fidèle au résultat de l'élection. \nLa Californie et le Nord-Est (qui sont souvent favorisés dans les statistiques précédentes) sont plutôt démocrates, alors que le Sud-Est, plus défavorisé, penche plutôt en faveur des républicains.")
print("De plus, si on s'intéresse uniquement à la taille des cercles, cette carte nous permet également de visualiser la densité de la population. Cette densité est beaucoup plus élevée sur la côte Ouest et sur la moitié Est du pays, au contraire du centre qui est relativement très peu densément peuplé et beaucoup plus rural.")

## Densité de population

In [ ]:
total_db_geo["pop_density_2013"] = total_db_geo["resident_pop_year_2013"] / total_db_geo["geometry"].area
total_db_geo.plot(aspect = 1, figsize = (15,15), column = "pop_density_2013", legend = True, scheme = "QUANTILES", k=4, 
linewidth = 0.1, legend_kwds={'loc': 'lower left'})
plt.title("Densité de population en 2013")
plt.show()

On remarque que l'Est et les côtes du pays concentrent les territoires les plus denses, alors que que l'Ouest et le centre ont des densités plus faibles

In [ ]:
print("On remarque que l'Est et les côtes du pays concentrent les territoires les plus denses, alors que que l'Ouest et le centre ont des densités plus faibles")

In [ ]:
sns.set(style="ticks")
x = total_db_geo["pop_density_2013"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution de la densité de population en 2013")
ax_hist.set_xlabel('Densité')

On remarque qu'il y a un outlier particulièrement surprenant, vérifions à la main ce qu'il est en est.

In [ ]:
print("On remarque qu'il y a un outlier particulièrement surprenant, vérifions à la main ce qu'il est en est.")

In [ ]:
total_db_geo[total_db_geo["pop_density_2013"] == total_db_geo["pop_density_2013"].max()]
print("Le comté de Fairfax semble présenter une erreur, nous allons regarder la distributioin de la densité sans les valeurs trop extrêmes.")

In [ ]:
sns.set(style="ticks")
x = total_db_geo[total_db_geo["pop_density_2013"] <  50000]["pop_density_2013"]

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution de la densité de population en 2013")
ax_hist.set_xlabel('Densité')

La distribution possède de nombreux outliers à droite, même quand on se restreint à des valeurs plus faibles.

## Comparaison des différentes années

Nous avons les données sur plusieurs années (entre 2013 et 2018, années de constitution de la base des incidents). Plutôt que faire une carte par année, nous allons vérifier que les données ne changent pas trop d'une année à l'autre, et que les tendances générales restent stables. Nous allons donc regarder les matrices de corrélation d'une même variable entre différentes années.

### Chômage

In [ ]:
print("\nII. Comparaison des différentes années\n")
print("Nous avons les données sur plusieurs années (entre 2013 et 2018, années de constitution de la base des incidents). \nPlutôt que faire une carte par année, nous allons vérifier que les données ne changent pas trop d'une année à l'autre, et que les tendances générales restent stables. Nous allons donc regarder les matrices de corrélation d'une même variable entre différentes années.\n")
print("Chômage :")

In [ ]:
matrix = total_db_geo[["unemp_rate_year_2013", "unemp_rate_year_2014", "unemp_rate_year_2015", 
"unemp_rate_year_2016", "unemp_rate_year_2017", "unemp_rate_year_2018"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation des taux de chômage", labels = {"color" : "Corrélation"})
fig.show()

On voit que toutes les corrélations sont supérieures à 80%, ce qui est relativement important et laisse penser que le taux de chômage reste relativement stable sur les années.

In [ ]:
print("On voit que toutes les corrélations sont supérieures à 80%, ce qui est relativement important et laisse penser que le taux de chômage reste relativement stable sur les années considérées.")

In [ ]:
sns.set_theme(style="ticks")
x = total_db_geo["unemp_rate_year_2018"]
y = total_db_geo["unemp_rate_year_2013"]
joint_plot = sns.jointplot(x=x, y=y, kind="hex", height = 5)
joint_plot.set_axis_labels("Taux de chômage en 2018", "Taux de chômage en 2013")
joint_plot.fig.suptitle("Hexbin du taux de chômage entre 2013 et 2018", y =1)

Cet hexbin montre que les taux de chômage entre 2013 et 2018 sont effectivement positivement corrélés. On remarque également qu'il n'y a que très peu de comtés ayant un faible taux à une année et un taux important pour l'autre année, ce qui suggère que les analyses resteront valables peu importe l'année retenue pour cette variable. Il convient de vérifier cela pour les autres variables.

### Revenu annuel médian par foyer

In [ ]:
print("Cet hexbin montre que les taux de chômage entre 2013 et 2018 sont effectivement positivement corrélés. \nOn remarque également qu'il n'y a que très peu de comtés ayant un faible taux à une année et un taux important pour l'autre année, ce qui suggère que les analyses resteront valables peu importe l'année retenue pour cette variable. \nIl convient de vérifier cela pour les autres variables.")
print("\nRevenu annuel médian par foyer :")

In [ ]:
matrix = total_db_geo[["med_h_income_year_2013", "med_h_income_year_2014", "med_h_income_year_2015", 
"med_h_income_year_2016", "med_h_income_year_2017", "med_h_income_year_2018"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation des revenues médians", labels = {"color" : "Corrélation"})
fig.show()

In [ ]:
sns.set_theme(style="ticks")
x = total_db_geo["med_h_income_year_2018"]
y = total_db_geo["med_h_income_year_2013"]
joint_plot = sns.jointplot(x=x, y=y, kind="hex")
joint_plot.set_axis_labels("Revenu médian en 2018", "Revenu médian en 2013")
joint_plot.fig.suptitle("Hexbin du revenu annuel médian par foyer entre 2013 et 2018", y = 1)

On observe la même chose que pour le chômage : les corrélations sont mêmes plus importantes puisque toutes supérieures à 95%. Le hexbin montre également cette quasi parfaite corrélation entre les deux années extrêmes, car tous les comtés sont situés sur la diagonale. Il y a donc très peu de comtés qui sont passés d'un revenu médian important à un revenu médian faible (ou l'inverse).

### Proportion de détenteurs d'un bachelor

In [ ]:
print("On observe la même chose que pour le chômage : les corrélations sont mêmes plus importantes puisque toutes supérieures à 95%. \nLe hexbin montre également cette quasi parfaite corrélation entre les deux années extrêmes, car tous les comtés sont situés sur la diagonale. Il y a donc très peu de comtés qui sont passés d'un revenu médian important à un revenu médian faible (ou l'inverse).")
print("\nProportion de détenteurs d'un bachelor :")

In [ ]:
matrix = total_db_geo[["bachelors_deg_year_2013", "bachelors_deg_year_2014", "bachelors_deg_year_2015", 
"bachelors_deg_year_2016", "bachelors_deg_year_2017", "bachelors_deg_year_2018"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation des parts de détenteurs de bachelor", labels = {"color" : "Corrélation"})
fig.show()

In [ ]:
sns.set_theme(style="ticks")
x = total_db_geo["bachelors_deg_year_2018"]
y = total_db_geo["bachelors_deg_year_2013"]
joint_plot = sns.jointplot(x=x, y=y, kind="hex")
joint_plot.set_axis_labels("Détenteurs d'un bachelor en 2018", "Détenteurs d'un bachelor en en 2013")
joint_plot.fig.suptitle("Hexbin de la part de détenteurs d'un bachelor entre 2013 et 2018", y =1)

Encore une fois, les données suggèrent qu'utiliser une année ou l'autre n'aura pas d'influence sur les résultats de l'analyse du fait de la similarité des années.

### Bénéficaires du programme SNAP

In [ ]:
print("Encore une fois, les données suggèrent qu'utiliser une année ou l'autre n'aura pas d'influence sur les résultats de l'analyse du fait de la similarité des années.")
print("\nBénéficaires du programme SNAP :")

In [ ]:
total_db_geo["snap_share_year_2013"] = total_db_geo["snap_beneficiaries_year_2013"] / total_db_geo["resident_pop_year_2013"] / 1000
total_db_geo["snap_share_year_2014"] = total_db_geo["snap_beneficiaries_year_2014"] / total_db_geo["resident_pop_year_2014"] / 1000
total_db_geo["snap_share_year_2015"] = total_db_geo["snap_beneficiaries_year_2015"] / total_db_geo["resident_pop_year_2015"] / 1000
total_db_geo["snap_share_year_2016"] = total_db_geo["snap_beneficiaries_year_2016"] / total_db_geo["resident_pop_year_2016"] / 1000
total_db_geo["snap_share_year_2017"] = total_db_geo["snap_beneficiaries_year_2017"] / total_db_geo["resident_pop_year_2017"] / 1000
total_db_geo["snap_share_year_2018"] = total_db_geo["snap_beneficiaries_year_2018"] / total_db_geo["resident_pop_year_2018"] / 1000


In [ ]:
matrix = total_db_geo[["snap_share_year_2013", "snap_share_year_2014", "snap_share_year_2015", 
"snap_share_year_2016", "snap_share_year_2017", "snap_share_year_2018"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation des parts de bénéficiaires du SNAP", labels = {"color" : "Corrélation"})
fig.show()

In [ ]:
sns.set_theme(style="ticks")
x = total_db_geo["snap_share_year_2018"]
y = total_db_geo["snap_share_year_2013"]
joint_plot = sns.jointplot(x=x, y=y, kind="hex")
joint_plot.set_axis_labels("Bénéficaires du SNAP en 2018", "Bénéficaires du SNAP en 2013")
joint_plot.fig.suptitle("Hexbin de la part bénéficaires du SNAP entre 2013 et 2018", y =1)

Même si la diagonale est plus étalée en haut à droite (donc quand les valeurs sont relativement importantes), rien ne semble indiquer qu'il y a des différences significatives dans la structure des données entre ces deux années.

## Corrélation entre les variables

Nous allons arbitrairement prendre les variables de 2013. Nous avons vu précédemment que cela n'allait pas changer beaucoup les résultats des analyses.

In [ ]:
print("Même si la diagonale est plus étalée en haut à droite (donc quand les valeurs sont relativement importantes), rien ne semble indiquer qu'il y a des différences significatives dans la structure des données entre ces deux années.")
print("\nCorrélation entre les variables :\n")
print("Nous allons arbitrairement prendre les variables de 2013. Nous avons vu précédemment que cela n'allait pas changer beaucoup les résultats des analyses.")

In [ ]:
matrix = total_db_geo[["unemp_rate_year_2013", "snap_share_year_2013", "bachelors_deg_year_2013", "med_h_income_year_2013", "per_dem", "score_legis"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation de 2013", labels = {"color" : "Corrélation"})
fig.show()

Les corrélations sont logiques et relativement aisées à expliquer, sauf pour le score legislatif. Le taux de chômage est postiviment corrélé aux bénéficiaires de l'aide alimentaire (suggère des comtés paupérisés), négativement à la part de détenteurs d'un bachelor (les plus éduqués sont a priori moins au chômage que les autres), négativement au revenu médian (plus il y a de chômage, plus ce revenu est abs a priori), et positivement au vote démocrate (plus diffile à bien expliquer).

Globalement, les variables "positives" comme le revenu médian et la part de diplômés sont positivement corrélées entre elles et négativement corrélées avec les variables "négatives" comme la part de bénéficiaires de l'aide alimentaire et le taux de chômage. Le vote démocrate a la même corrélation avec toutes les variables (0.2), sauf pour la part de détenteurs d'un bachelor, qui est plus forte (0.4) : cela s'explique bien car les plus éduqués votent plus démocrates que les autres.

La corrélation la plus forte du score législatif (hormis avec lui-même) est avec le revenu médian et avec le vote démocrate. Peut-être que certains états pauvres votent démocrates et ont donc une législation plus forte sur les armes, mais il est difficile de tirer de réelles conclusions à partir de ces corrélations là.

# Liens de ces caractéristiques avec la violence par arme à feu

Maintenant que nous avons décrit plus en détail les comtés, nous allons esssayer de voir comment cela s'articule avec les données sur les violences par arme à feu. Commençons par la densité des incidents.

In [ ]:
print("Les corrélations sont logiques et relativement aisées à expliquer, sauf pour le score legislatif. \nLe taux de chômage est postiviment corrélé aux bénéficiaires de l'aide alimentaire (ce qui suggère des comtés paupérisés), négativement à la part de détenteurs d'un bachelor (les plus éduqués sont a priori moins au chômage que les autres), \nnégativement au revenu médian (plus il y a de chômage, plus ce revenu est bas a priori), et positivement au vote démocrate (plus diffile à bien expliquer).\n")
print("Globalement, les variables 'positives' comme le revenu médian et la part de diplômés sont positivement corrélées entre elles et négativement corrélées avec les variables 'négatives' comme la part de bénéficiaires de l'aide alimentaire et le taux de chômage. \nLe vote démocrate a la même corrélation avec toutes les variables (0.2), sauf pour la part de détenteurs d'un bachelor, qui est plus forte (0.4) : cela s'explique bien car les plus éduqués votent plus démocrates que les autres.\n")
print("La corrélation la plus forte du score législatif (hormis avec lui-même) est avec le revenu médian et avec le vote démocrate. \nPeut-être que certains états riches votent démocrates et ont donc une législation plus forte sur les armes, mais il est difficile de tirer de réelles conclusions à partir de ces corrélations là.\n")
print("\nIII. Liens de ces caractéristiques avec la violence par arme à feu\n")
print("Maintenant que nous avons décrit plus en détail les comtés, nous allons esssayer de voir comment cela s'articule avec les données sur les violences par arme à feu. \nCommençons par la densité des incidents :")

In [ ]:
#count of incidents for each county
incident_count_by_county = merge_geo.groupby('id_county')['incident_id'].count().reset_index() 

#new dataframe with the count
incident_pop_geo=pd.merge(incident_count_by_county,merge_geo, on='id_county', how='right')
incident_pop_geo.rename(columns={'incident_id_x':'incident_count_by_county'}, inplace=True)

#calcul de la densité
agg_incident_county = incident_pop_geo.groupby('id_county').agg({'incident_count_by_county': 'first', 'resident_pop_year_2016': 'first'}).reset_index()
agg_incident_county['incident_density'] = agg_incident_county['incident_count_by_county'] / agg_incident_county['resident_pop_year_2016'] / 1000


# Create a GeoDataFrame with the incident density
incident_density_geo = total_db_geo.merge(agg_incident_county, on='id_county', how='left')
incident_density_geo["incident_density"] = incident_density_geo["incident_density"] * 1000

# ajouter la valeur à 0 à ceux qui n'apparaissaient pas dans la base de gun violences
incident_density_geo["incident_density"] = incident_density_geo["incident_density"].fillna(0)
incident_density_geo["incident_count_by_county"] = incident_density_geo["incident_count_by_county"].fillna(0)

In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu", 
column = "incident_density")

plt.title("Densité d'incidents pour 1000 habitants")
plt.show()

Encore une fois, la carte est difficile à lire. Nous allons donc utiliser la même méthode que précédemment et regrouper les comtés dans 4 quantiles.

In [ ]:
print("Encore une fois, la carte est difficile à lire. Nous allons donc utiliser la même méthode que précédemment et regrouper les comtés dans 4 quantiles.")

In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu_r", 
column = "incident_density", scheme = "QUANTILES", k=4, legend_kwds={'loc': 'lower left'})

plt.title("Densité d'incidents pour 1000 habitants par comtés")
plt.show()

Le centre du pays semble avoir une assez faible densité d'incidents, contrairements à la côte Est. La côte Ouest est quant à elle plus contrastée.

In [ ]:
print("Le centre du pays semble avoir une assez faible densité d'incidents, contrairements à la côte Est. La côte Ouest est quant à elle plus contrastée.")

In [ ]:
incident_density_geo["state_resident_pop_year_2016"] = incident_density_geo.groupby("Etat")["resident_pop_year_2016_x"].transform("sum")
incident_density_geo["incident_count_by_state"] = incident_density_geo.groupby("Etat")["incident_count_by_county"].transform("sum")
incident_density_geo["incident_density_state"] = incident_density_geo["incident_count_by_state"] / incident_density_geo["state_resident_pop_year_2016"]

In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu_r", 
column = "incident_density_state", scheme = "QUANTILES", k=4, legend_kwds={'loc': 'lower left'})

plt.title("Densité d'incidents pour 1000 habitants par états")
plt.show()

Cette carte, quoique moins précise, est plus lisible que la précédente. On y voit que l'Ouest est globalement moins touché par les incidents à arme à fau que l'Est, qui concentre les états avec la plus grande densité.

In [ ]:
print("Cette carte, quoique moins précise, est plus lisible que la précédente. \nOn y voit que l'Ouest est globalement moins touché par les incidents à arme à feu que l'Est, qui concentre les états avec la plus grande densité d'incidents.")

In [ ]:
sns.set(style="ticks")
x = incident_density_geo["incident_count_by_county"] 

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution du nombre d'incidents par comtés")
ax_hist.set_xlabel("Nombre d'incidents")

On voit que la majorité des comtés ont moins de 500 incidents. Le graphique est cependant peu lisible à cause des importants outliers qui l'étirent. Après vérification manuelle, le comté avec le plus d'incident est le comté de Cook, dans l'Illinois. Cela se comprend assez bien car la ville de Chicago, réputée pour sa criminalité élevée, en est le chef-lieu.

In [ ]:
print("On remarque que la majorité des comtés enregistre moins de 500 incidents. Le graphique est cependant peu lisible à cause des importants outliers qui l'étirent. \nAprès vérification manuelle, le comté ayant le plus d'incident est le comté de Cook, dans l'Illinois. Cela se comprend assez bien car la ville de Chicago, réputée pour sa criminalité élevée, en est le chef-lieu.")

In [ ]:
sns.set(style="ticks")
x = incident_density_geo[incident_density_geo["incident_count_by_county"] < 1000]["incident_count_by_county"]

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution du nombre d'incidents par comtés pour les comtés avec moins de 1000 incidents")
ax_hist.set_xlabel("Nombre d'incidents")

Même en ne prenant que les comtés avec moins de 1000 incidents la distribution garde la même forme : la majorité des comtés sont tout à gauche de la distribution (donc avec peu d'incidents) et de nombreux outliers atteignent des valeurs très importantes. Ils se voient sur le box plot mais pas sur l'histogramme.

In [ ]:
print("Même en ne prenant que les comtés avec moins de 1000 incidents la distribution garde la même forme : la majorité des comtés est tout à gauche de la distribution (donc avec peu d'incidents) et de nombreux outliers atteignent des valeurs très importantes. \nOn les remarque sur le box plot mais pas sur l'histogramme.")

In [ ]:
# comptage du nombre de suicides par comtés
# on fait la somme des indicatrices suicides après avoir groupé par comtés
incident_density_geo = incident_density_geo.merge(merge_geo.groupby('id_county')["indic_suicide"].sum(), on = "id_county", how = "left")
incident_density_geo = incident_density_geo.rename(columns = {"indic_suicide" : "suicide_by_county"})
incident_density_geo["suicide_by_county"] = incident_density_geo["suicide_by_county"].fillna(0)
incident_density_geo["suicide_density"] = incident_density_geo["suicide_by_county"] / incident_density_geo["resident_pop_year_2016_x"]

In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu_r", 
column = "suicide_density", scheme = "QUANTILES", k=4, legend_kwds={'loc': 'lower left'})

plt.title("Densité de suicides pour 1000 habitants par comtés")
plt.show()

A l'Ouest, les grands comtés (donc les plus ruraux) semblent être les plus touchés. Le centre du pays semble relativement moins touché. A l'Est, il est difficile de décrire quelque chose de précis, si ce n'est que la Géorgie et les deux Caroline paraissent être plus touchées que la moyenne.

In [ ]:
print("A l'Ouest, les grands comtés (donc les plus ruraux) semblent être les plus touchés. Le centre du pays semble relativement moins touché. \nA l'Est, il est difficile de décrire quelque chose de précis, si ce n'est que la Géorgie et les deux Caroline paraissent être plus touchées que la moyenne.")

In [ ]:
sns.set(style="ticks")
x = incident_density_geo["suicide_by_county"]

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10,5))

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=30, ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

median_value = x.median()
ax_hist.axvline(median_value, color='red', linestyle='dashed', linewidth=2, label='Median')

plt.suptitle("Distribution du nombre de suicides par comtés")
ax_hist.set_xlabel("Nombre d'incidents")

On observe la même forme de distribution que pour les incidents. La seule différence est que tout semble diviser par 10, ce qui est logique puisque les suicides représentent environ 13% des incidents.

Intéressons nous désormais au nombre de tués et de blessés.

In [ ]:
print("On observe la même forme de distribution que pour les incidents. La seule différence est que tout semble diviser par 10, ce qui est logique puisque les suicides représentent environ 13% des incidents.")
print("\nIntéressons nous désormais au nombre de tués et de blessés.")

In [ ]:
# calcul du nombre de tués par county
incident_density_geo = incident_density_geo.merge(merge_geo.groupby('id_county')["n_killed"].sum(), on = "id_county", how = "left")
incident_density_geo = incident_density_geo.rename(columns = {"n_killed" : "n_killed_by_county"})
incident_density_geo["n_killed_by_county"] = incident_density_geo["n_killed_by_county"].fillna(0)
incident_density_geo["n_killed_density"] = incident_density_geo["n_killed_by_county"] / incident_density_geo["resident_pop_year_2016_x"]

# calcul du nombre de blessés par county
incident_density_geo = incident_density_geo.merge(merge_geo.groupby('id_county')["n_injured"].sum(), on = "id_county", how = "left")
incident_density_geo = incident_density_geo.rename(columns = {"n_injured" : "n_injured_by_county"})
incident_density_geo["n_injured_by_county"] = incident_density_geo["n_injured_by_county"].fillna(0)
incident_density_geo["n_injured_density"] = incident_density_geo["n_injured_by_county"] / incident_density_geo["resident_pop_year_2016_x"]


In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu_r", 
column = "n_killed_density", scheme = "QUANTILES", k=4, legend_kwds={'loc': 'lower left'})

plt.title("Densité de tués pour 1000 habitants par comtés")
plt.show()

La distribution reste relativement proche de celles des incidents. Le centre paraît moins touchés par les victimes que les incidents, mais c'est la seule différence vraiment visible.

In [ ]:
print("La distribution reste relativement proche de celles des incidents. Le centre paraît moins touchés par les victimes que les incidents, mais c'est la seule différence vraiment visible.")

In [ ]:
incident_density_geo.plot(aspect = 1, figsize = (12,6), legend = True, linewidth = 0.1, cmap = "RdBu_r", 
column = "n_injured_density", scheme = "QUANTILES", k=4, legend_kwds={'loc': 'lower left'})

plt.title("Densité de blessés pour 1000 habitants par comtés")
plt.show()

La distribution est très similaire à celles des tués.

Corrélation avec les variables sociaux démographiques.

In [ ]:
print("La distribution est très similaire à celles des tués.\n")
print("Corrélation avec les variables sociaux démographiques :")

In [ ]:
#ajout de la densité de population au dataframe des incidents
incident_density_geo = incident_density_geo.merge(total_db_geo[["pop_density_2013", "id_county"]], on = "id_county")

In [ ]:
matrix = incident_density_geo[["unemp_rate_year_2013", "snap_share_year_2013", "bachelors_deg_year_2013", 
"med_h_income_year_2013", "per_dem", "score_legis", "pop_density_2013", "incident_density", "suicide_density", "n_killed_density", "n_injured_density"]].corr()
fig = px.imshow(matrix, title= "Matrice de corrélation de 2013", labels = {"color" : "Corrélation"})
fig.show()

La densité d'incidents ne semble être corrélées de façon significative qu'avec les autres variables dépendants des incidents. On peut cependant remarquer la corrélation positive avec le vote démocrate et avec la part de bénéficaires de l'aide alimentaire.

On observe le même phénomène pour les trois autres variables des incidents (densité de suicide, de morts et de blessés). Cela semble donc indiquer que les variables sociaux démographiques étudiées précédemment ne permettent pas vraiment d'expliquer l'apparition des incidents et de leurs victimes.

In [ ]:
print("La densité d'incidents ne semble être corrélée de façon significative qu'avec les autres variables dépendants des incidents. \nOn peut cependant remarquer la corrélation positive avec le vote démocrate et avec la part de bénéficaires de l'aide alimentaire.")
print("On observe le même phénomène pour les trois autres variables des incidents (densité de suicide, de morts et de blessés). \nCela semble donc indiquer que les variables sociaux démographiques étudiées précédemment ne permettent pas vraiment d'expliquer l'apparition des incidents et de leurs victimes.")

In [ ]:
matrix = incident_density_geo[["unemp_rate_year_2013", "snap_share_year_2013", "bachelors_deg_year_2013", 
"med_h_income_year_2013", "per_dem", "score_legis", "incident_density", "pop_density_2013"]]
scatter = pd.plotting.scatter_matrix(matrix, figsize = (30,30))

On voit sur la dernière ligne que la densité des incidents semblent pouvoir n'être simplement prédite à partir d'aucune des variables sélectionnées précédemment.

On observe la même chose pour la densité de tués, blessés et suicides.

In [ ]:
print("On voit sur la dernière ligne que la densité des incidents semble pouvoir n'être prédite simplement à partir d'aucune des variables sélectionnées précédemment.")
print("On observe la même chose pour la densité de tués, blessés et suicides.")